In [2]:
import requests
from bs4 import BeautifulSoup
import time
import random
import pandas as pd

In [ ]:
base_url = 'https://mogi.vn/ha-noi/mua-nha-dat'
respone = requests.get(base_url)


In [26]:
Links = []
for page in range(1,21):
    url = f"{base_url}?&cp={page}"
    print(f"Đang cào trang {page}: {url}")
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        # Tìm tất cả thẻ a có class="link-overlay"
        links = soup.find_all('a', class_='link-overlay')

        # In ra href của từng thẻ
        for link in links:
            href = link.get('href')
            Links.append(href)
            # print(href)
    else:
        print("Không thể truy cập trang:", response.status_code)

 

Đang cào trang 1: https://mogi.vn/ha-noi/mua-nha-dat?&cp=1
Đang cào trang 2: https://mogi.vn/ha-noi/mua-nha-dat?&cp=2
Đang cào trang 3: https://mogi.vn/ha-noi/mua-nha-dat?&cp=3
Đang cào trang 4: https://mogi.vn/ha-noi/mua-nha-dat?&cp=4
Đang cào trang 5: https://mogi.vn/ha-noi/mua-nha-dat?&cp=5
Đang cào trang 6: https://mogi.vn/ha-noi/mua-nha-dat?&cp=6
Đang cào trang 7: https://mogi.vn/ha-noi/mua-nha-dat?&cp=7
Đang cào trang 8: https://mogi.vn/ha-noi/mua-nha-dat?&cp=8
Đang cào trang 9: https://mogi.vn/ha-noi/mua-nha-dat?&cp=9
Đang cào trang 10: https://mogi.vn/ha-noi/mua-nha-dat?&cp=10
Đang cào trang 11: https://mogi.vn/ha-noi/mua-nha-dat?&cp=11
Đang cào trang 12: https://mogi.vn/ha-noi/mua-nha-dat?&cp=12
Đang cào trang 13: https://mogi.vn/ha-noi/mua-nha-dat?&cp=13
Đang cào trang 14: https://mogi.vn/ha-noi/mua-nha-dat?&cp=14
Đang cào trang 15: https://mogi.vn/ha-noi/mua-nha-dat?&cp=15
Đang cào trang 16: https://mogi.vn/ha-noi/mua-nha-dat?&cp=16
Đang cào trang 17: https://mogi.vn/ha-noi/

In [61]:
import csv
Link_list = []
data_list = []
with open('mogi_links.csv', 'a', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['link'])

    for page in range(380, 400):
        url = f"{base_url}?&cp={page}"
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")
        links = soup.find_all("a", class_="link-overlay")
        for link in links:
            href = link.get("href")
            if href:
                writer.writerow([href])
                Link_list.append(href)


for link in Link_list:
    res = requests.get(link)
    if res.status_code == 200:
        data = {}
        sop = BeautifulSoup(res.text,"html.parser")
        infos = sop.find_all("div", class_ = "info-attr clearfix")
        price = sop.find('div',class_ = "price")
        address = sop.find('div',class_ = "address")
        data['price'] = price.get_text(strip = True)
        data['address'] = address.get_text(strip = True)
        for info in infos:
            span = info.find_all('span')
            if len(span) >= 2:
                key = span[0].get_text(strip=True)
                value = span[1].get_text(strip=True)
                data[key] = value
        data_list.append(data)
        # time.sleep(random.uniform(1,2))

fieldnames = set()
for d in data_list:
    fieldnames.update(d.keys())
fieldnames = list(fieldnames)

with open('batdongsan.csv', 'a', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(data_list)

print("Đã lưu dữ liệu vào file batdongsan.csv")

Đã lưu dữ liệu vào file batdongsan.csv


In [36]:
df = pd.read_csv("batdongsan.csv")
print(df.head())

  Diện tích sử dụng    Mã BĐS           price Pháp lý   Ngày đăng  Nhà tắm  \
0            100 m2  22712100  5 tỷ 800 triệu   Sổ đỏ  15/10/2025      1.0   
1             80 m2  22712094  4 tỷ 350 triệu   Sổ đỏ  15/10/2025      2.0   
2             90 m2  22711721  4 tỷ 580 triệu   Sổ đỏ  15/10/2025      2.0   
3            120 m2  22711722  4 tỷ 700 triệu   Sổ đỏ  15/10/2025      2.0   
4            100 m2  22711723  5 tỷ 400 triệu   Sổ đỏ  15/10/2025      2.0   

   Phòng ngủ                                            address Diện tích đất  
0        2.0  Đại La, Phường Đồng Nhân, Quận Hai Bà Trưng, H...           NaN  
1        2.0  Thanh Nhàn, Phường Thanh Nhàn, Quận Hai Bà Trư...           NaN  
2        3.0  Nhân Hòa, Phường Nhân Chính, Quận Thanh Xuân, ...           NaN  
3        3.0  Thanh Nhàn, Phường Thanh Nhàn, Quận Hai Bà Trư...           NaN  
4        3.0  Yên Lạc, Phường Vĩnh Tuy, Quận Hai Bà Trưng, H...           NaN  


In [58]:
df = pd.read_csv("batdongsan.csv")
duplicate = df.duplicated()
print(f"Số lượng dòng trùng lặp: {duplicate.sum()}")


Số lượng dòng trùng lặp: 7


Số dòng ban đầu: 4492
Số dòng sau khi loại bỏ trùng lặp: 4488
Số dòng trùng lặp đã xóa: 4
Tỷ lệ trùng lặp: 0.09%
